# Multi-Processing-Beispiel

Wir beginnen hier mit Code, der klar und einfach ist und von oben nach unten ausgeführt wird. Er ist einfach zu entwickeln und inkrementell zu testen:

In [1]:
import urllib.request
from multiprocessing.pool import ThreadPool as Pool

sites = [
    'https://jupyter-tutorial.readthedocs.io/de/latest/',
    'https://github.com/veit/jupyter-tutorial/',
    'https://www.cusy.io/de',
]

def sitesize(url):
    ''' Determine the size of a website '''
    with urllib.request.urlopen(url) as u:
        page = u.read()
        return url, len(page)

pool = Pool(10)
for result in pool.imap_unordered(sitesize, sites):
    print(result)

('https://www.cusy.io/de', 15655)
('https://jupyter-tutorial.readthedocs.io/de/latest/', 12630)
('https://github.com/veit/jupyter-tutorial/', 98527)


> **Hinweis 1:** Eine gute Entwicklungsstrategie ist die Verwendung von [map](https://docs.python.org/3/library/functions.html#map), um den Code in einem einzelnen Prozess und einem einzelnen Thread zu testen, bevor zu Multi-Processing gewechselt wird.

> **Hinweis 2:** Um besser einschätzen zu können, wann `ThreadPool` und wann Prozess-`Pool` verwendet werden sollte, hier einige Faustregeln:
> 
> * `multiprocessing.pool.ThreadPool` sollte für IO-lastige Jobs verwendet werden.
> * `multiprocessing.Pool` sollte für CPU-lastige Jobs verwendet werden.
> * Für CPU- und IO-lastige Jobs bevorzuge ich üblicherweise `multiprocessing.Pool`, da hierdurch eine bessere Prozessisolierung erreicht wird.
> * Bei Python 3 schaut euch auch die Pool-Implementierung von [concurrent.future.Executor](https://docs.python.org/3/library/concurrent.futures.html?highlight=concurrent%20futures#concurrent.futures.Executor) an.

In [2]:
import urllib.request
from multiprocessing.pool import ThreadPool as Pool

sites = [
    'https://jupyter-tutorial.readthedocs.io/de/latest/',
    'https://github.com/veit/jupyter-tutorial/',
    'https://www.cusy.io/de',
]

def sitesize(url):
    ''' Determine the size of a website '''
    with urllib.request.urlopen(url) as u:
        page = u.read()
        return url, len(page)

for result in map(sitesize, sites):
    print(result)

('https://jupyter-tutorial.readthedocs.io/de/latest/', 12630)
('https://github.com/veit/jupyter-tutorial/', 98651)
('https://www.cusy.io/de', 15655)


## Was ist parallelisierbar?

### Amdahlsche Gesetz

> Der Geschwindigkeitszuwachs vor allem durch den sequentiellen Anteil des Problems beschränkt, da sich dessen Ausführungszeit durch Parallelisierung nicht verringern lässt. Zudem entstehen durch Parallelisierung zusätzliche Kosten wie etwa für die Kommunikation und die Synchronisierung der Prozesse.

In unserem Beispiel können die folgenden Aufgaben nur seriell abgearbeitet werden:

* UDP DNS request für die URL
* UDP DNS response
* Socket vom OS
* TCP-Connection
* Senden des HTTP Request für die Root-Ressource
* Warten auf die TCP Response
* Zählen der Zeichen auf der Website

In [3]:
import urllib.request
from multiprocessing.pool import ThreadPool as Pool

sites = [
    'https://jupyter-tutorial.readthedocs.io/de/latest/',
    'https://github.com/veit/jupyter-tutorial/',
    'https://www.cusy.io/de',
]

def sitesize(url):
    ''' Determine the size of a website '''
    with urllib.request.urlopen(url) as u:
        page = u.read()
        return url, len(page)

pool = Pool(10)
for result in pool.imap_unordered(sitesize, sites):
    print(result)

('https://www.cusy.io/de', 15655)
('https://jupyter-tutorial.readthedocs.io/de/latest/', 12630)
('https://github.com/veit/jupyter-tutorial/', 98526)


> **Hinweis:** [imap_unordered](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.pool.Pool.imap_unordered) wird verwendet, um die Reaktionsfähigkeit zu verbessern. Dies ist jedoch nur möglich, da die Funktion das Argument und das Ergebnis als Tuple zurückgibt.

## Tipps

* Macht nicht zu viele Trips hin und her

   Erhaltet ihr zu viele iterierbare Ergebnisse, ist dies ein guter Indikator für zu viele Trips, wie z.B. in

        def sitesize(url, start):
            req = urllib.request.Request()
            req.add_header('Range:%d-%d' % (start, start+1000))
            u = urllib.request.urlopen(url, req)
            block = u.read()
            return url, len(block)

* Macht auf jedem Trip relevante Fortschritte

   Sobald ihr den Prozess erhaltet, solltet ihr deutliche Fortschritte erzielen und euch nicht verzetteln. Das folgende Beispiel verdeutlicht zu kleine Zwischenschritte:

        def sitesize(url, results):
            with urllib.request.urlopen(url) as u:
                while True:
                    line = u.readline()
                    results.put((url, len(line)))

* Sendet und empfangt nicht zu viele Daten

   Das folgende Beispiel erhöht unnötig die Datenmenge:

        def sitesize(url):
            u = urllib.request.urlopen(url)
            page = u.read()
            return url, page